In this notebook, we present the accuracy of a further variation of the naive Bayes classifier introduced by [Rennie, Shih, Teevan, and Karger](https://people.csail.mit.edu/jrennie/papers/icml03-nb.pdf) which is implemented in sklearn as [``ComplementNB``](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html).  The cell below we show that this model outperforms both the Bernoulli and multinomial variants of the naive Bayes classifier on the task of text classification for the [20 Newsgroups dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) (although we are just using these models out-of-the-box and not tuning the smoothing parameter or modifying the preprocessing).  We then offer a derivation of the complement naive Bayes classifier by way of the one-versus-rest transformation to the multinomial naive Bayes classifier in the binary case.    

In [7]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.multiclass import OneVsRestClassifier

twenty_train = fetch_20newsgroups(subset='train',
                                  shuffle=True, 
                                  random_state=42)

twenty_test = fetch_20newsgroups(subset='test',
                                 shuffle=True, 
                                 random_state=42)

X_train = twenty_train.data
y_train = twenty_train.target
X_test  = twenty_test.data
y_test  = twenty_test.target

models = [BernoulliNB(),
          MultinomialNB(),
          ComplementNB(),
          ComplementNB(norm=True),
          OneVsRestClassifier(BernoulliNB()),
          OneVsRestClassifier(MultinomialNB()),
          OneVsRestClassifier(ComplementNB())] # It does not really make sense to do this lasat one.

for model in models:
    clf = Pipeline([
        ('vect', CountVectorizer()),
        ('clf', model)
    ])
    clf.fit(X_train, y_train)
    print(f'accuracy of {model} = {np.mean(clf.predict(X_test) == y_test)}')


accuracy of BernoulliNB() = 0.6307753584705258
accuracy of MultinomialNB() = 0.7728359001593202
accuracy of ComplementNB() = 0.8267392458842273
accuracy of ComplementNB(norm=True) = 0.8080191184280403
accuracy of OneVsRestClassifier(estimator=BernoulliNB()) = 0.6583908656399363
accuracy of OneVsRestClassifier(estimator=MultinomialNB()) = 0.7931492299522039
accuracy of OneVsRestClassifier(estimator=ComplementNB()) = 0.7930164630908125


In [4]:


import numpy as np

from sklearn.svm import SVC
X = np.array([
    [10, 10],
    [8, 10],
    [-5, 5.5],
    [-5.4, 5.5],
    [-20, -20],
    [-15, -20]
])
y = np.array([0, 0, 1, 1, 2, 2])
clf = OneVsRestClassifier(SVC()).fit(X, y)
clf.predict([[-19, -20], [9, 9], [-5, 5]])


array([2, 0, 1])

Here we give a brief derivation of the complement naive Bayes classifier assuming familiarity with the usual multinomial naive Bayes classifier.  Many classifiers take the form 
$$
h(x) = \arg\max_c f_c(x)
$$
where the functions $f_c(x)$ are called discrinant functions.  In the case of a binary classification problem, the classes are often labeled with $+$ and $-$ in which case, 
$$
h(x) = \arg\max_{\ast \in \{+,-\}} f_{\ast}(x) = \text{sign}(f_{+}(x) - f_{-}(x))
$$

Some learning algorithms (for example support vector machines) only apply directly to binary classification problems.  One way of obtaining a clasifier for a multiclass problem using such binary classifiers is from the [one-versus-rest](https://en.wikipedia.org/wiki/Multiclass_classification) transfromation to binary.  Assume that we have a response variable $C$ taking on classes $c_1,...,c_K$.  Suppose that we pick a class $c$ and identify all other classes together as another class $\tilde{c}$.  We then train a binary classifier on the data with the labelings $c$ and $\tilde{c}$ that returns a discriminant function $f_c(x)$ that when, positive indicates that the binary classifier believes we are in class $c$ with maginitude indicating degree of certainty, and similarly with negative values of the discrimant meaning the binary classifier believes we are not in $c$.  Then by using this learning algorithm on each class $c$, we obtain a discriminant for each class $f_c$ and then, the one-versus-rest classifier is given as 
$$
h_{\text{one-versus-rest}}(x) = \arg\max_{c} f_c(x)
$$

The binary classification rule we use is the mulitnomial naive Bayes classifier where now, instead of the assumption that the features $X$ conditioned on each class follow a multinomial distribution, we now make the stronger assumption that the features $X$ conditioned on any class or its complement follow a multinomial distribution.  The discriminant functions that result (after the usual application of Bayes' rule, removing terms completely determined by $X$ and taking the logarithm) is 
$$
f_c(x) = \log(P(C=c)) + \sum_i x_i \log(p_{ci}) - \log(1 - P(C=c)) - \sum_i x_i \log(p_{\tilde{c}i})
$$
where the terms $p_{ci}$ and $p_{\tilde{c}i}$ are the parameters of the respective multinomial distributions.  

We note that the term $\log(1- P(C=c))$ is not included by [Rennie, Shih, Teevan, and Karger](https://people.csail.mit.edu/jrennie/papers/icml03-nb.pdf) (equation 7).  This is harmless as this number is typically very small.  Indeed ignoring all of the prior terms (sometimes called using a uniform prior) typically makes little difference as many of the $p_{ci}$ are small and so the sums dominate.  

The complement naive Bayes classifier is then derived by also ignoring the "in-class" terms $\sum_i x_i \log(p_{ci})$ in the discriminant.  One possible explanation for the improvement of performance by the complement naive bayes model is the estimates of the $p_{tilde{c}i}$ parameters are typically done using more data than the $p_{ci}$ and therefore complement naive Bayes might have less of an issue with bias.  